In [1]:
from rchess import Board
import numpy as np

In [1]:
from model import load_as_trt_model
import numpy as np
trt_func, model = load_as_trt_model()


from rchess import Board
import time

2024-12-30 15:32:27.406024: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-30 15:32:28.853873: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5773 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5
2024-12-30 15:32:29.455575: E tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:87] DefaultLogger 3: [runtime.cpp::~Runtime::346] Error Code 3: API Usage Error (Parameter check failed at: runtime/rt/runtime.cpp::~Runtime::346, condition: mEngineCounter.use_count() == 1. Destroying a runtime before destroying deserialized engines created by the runtime leads to undefined behavior.
)


In [17]:
from mcts import find_best_move

times = []
for i in range(1):
    board = Board()
    root = None
    for i in range(5):
        t1 = time.time()
        move, root, child_visits = find_best_move(board, root, trt_func, 800)
        t2 = time.time()
        times.append(t2 - t1)
        board.push_num(move)
        terminal, winner = board.terminal()
        if terminal:
            print("Game over, winner is", winner)
        root = root[move]
print("Average time", np.mean(times))


Average time 1.8448209762573242


In [1]:
from model import load_as_trt_model
import numpy as np
trt_func, model = load_as_trt_model()

from rchess import Board
import time

2024-12-30 15:46:10.786818: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-30 15:46:12.199576: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5772 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5
2024-12-30 15:46:12.795895: E tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:87] DefaultLogger 3: [runtime.cpp::~Runtime::346] Error Code 3: API Usage Error (Parameter check failed at: runtime/rt/runtime.cpp::~Runtime::346, condition: mEngineCounter.use_count() == 1. Destroying a runtime before destroying deserialized engines created by the runtime leads to undefined behavior.
)


In [10]:
import pstats, cProfile
from mcts import find_best_move
#import mcts
board = Board()
cProfile.runctx("find_best_move(board, None, trt_func, 800)", globals(), locals(), "Profile.prof")

s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Mon Dec 30 15:49:28 2024    Profile.prof

         239333 function calls (233256 primitive calls) in 0.169 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      801    0.027    0.000    0.038    0.000 c.pyx:231(evaluate_node)
       51    0.021    0.000    0.021    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_Execute}
     2032    0.018    0.000    0.026    0.000 c.pyx:190(select_child)
       51    0.012    0.000    0.074    0.001 c.pyx:178(make_predictions)
       51    0.011    0.000    0.011    0.000 constant_op.py:75(convert_to_eager_tensor)
      801    0.011    0.000    0.017    0.000 c.pyx:248(expand_node)
      2/1    0.010    0.005    0.170    0.170 c.pyx:58(find_best_move)
     1607    0.004    0.000    0.004    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    45486    0.003    0.000    0.003    0.000 c.pyx:18(__get__)
    17795    0.002    0.000    0.002    0.000 c.pyx:41(__setitem__)
      80

In [6]:
print(board.to_string())


  +---+---+---+---+---+---+---+---+
  | r |   | b | q | k | b |   | r | 8
  +---+---+---+---+---+---+---+---+
  | p | p | p | p | p | p | p | p | 7
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 5
  +---+---+---+---+---+---+---+---+
  |   |   |   |   | n |   |   |   | 4
  +---+---+---+---+---+---+---+---+
  | P |   | N |   |   | N |   |   | 3
  +---+---+---+---+---+---+---+---+
  |   | P | Q | P | P | P | P | P | 2
  +---+---+---+---+---+---+---+---+
  | R |   | B | K |   | B |   | R | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: r1bqkb1r/pppppppp/8/8/4n3/P1N2N2/1PQPPPPP/R1BK1B1R w kq - 3 6
Key: ef69a98e93ec1388


In [2]:
from mcts import SelfplayGame
times = []
for i in range(10):
    game = SelfplayGame()
    for i in range(10):
        t1 = time.time()
        move, child_visits = game.simulate(trt_func, 800)
        t2 = time.time()
        times.append(t2 - t1)
        game.make_move(move)
        terminal, winner = game.board.terminal()
        if terminal:
            print("Game over, winner is", winner)
print("Average time", np.mean(times))

Average time 0.31473344564437866


In [27]:
t1 = time.time()
move, child_visits = game.simulate(trt_func, 800)
t2 = time.time()
print(t2-t1)
game.make_move(move)
terminal, winner = game.board.terminal()
if terminal:
    print("Game over, winner is", winner)

0.4044685363769531


In [16]:
game = SelfplayGame()
t1 = time.time()
while True:
    move, child_visits = game.simulate(trt_func, 800)
    game.make_move(move)
    terminal, winner = game.board.terminal()
    if terminal:
        break
if winner == 1:
    print("White wins")
elif winner == -1:
    print("Black wins")
else:
    print("Draw")
t2 = time.time()
print(game.board.to_string())
print(t2-t1)
print(move)


Draw

  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 8
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 7
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 5
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 4
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   | Q |   |   | 3
  +---+---+---+---+---+---+---+---+
  |   |   | K |   |   |   |   |   | 2
  +---+---+---+---+---+---+---+---+
  |   |   |   |   | k |   |   |   | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: 8/8/8/8/8/5Q2/2K5/4k3 b - - 66 199
Key: d554af9f066be6d5
134.74590635299683
b1c2


In [4]:
import numpy as np
import timeit
import time
from math import sqrt, log

class Node:
    @property
    def Q(self):
        return self.W / self.N if self.N > 0 else 0.0

    def __init__(self):
        self.children = {}
        self.vloss = 0
        self.to_play = None

        # Values for MCTS
        self.N = 0
        self.W = 0.0
        self.P = 0.0

    def __getitem__(self, move: str):
        return self.children[move]
    
    def __setitem__(self, move: str, node):
        self.children[move] = node

    def is_leaf(self):
        return not self.children
    
# Create a dummy node with 30 fake children
dummy_node = Node()
dummy_node.N = 100  # Set parent visit count

for i in range(30):
    child = Node()
    child.N = np.random.randint(1, 100)  # Random visit count for each child
    child.W = np.random.rand() * 100  # Random total value for each child
    child.P = np.random.rand()  # Random prior probability for each child
    dummy_node.children[f"move_{i}"] = child

In [7]:
def select_child(node: Node, pb_c_base: float, pb_c_init: float, pb_c_factor: float, fpu: float):
    bestucb = -np.inf
    bestmove = None
    bestchild = None

    parent_N_sqrt = sqrt(node.N + 1)
    pb_c = log((node.N + pb_c_base + 1) / pb_c_base) * pb_c_factor + pb_c_init

    for move, child in node.children.items():
        if child.N == 0:
            puct = child.Q + pb_c * child.P * parent_N_sqrt / (child.N + 1)
        else:
            puct = fpu
        if puct > bestucb:
            bestucb = puct
            bestmove = move
            bestchild = child

    return bestmove, bestchild

# Create a dummy node with 30 fake children
dummy_node = Node()
dummy_node.N = 100  # Set parent visit count

for i in range(30):
    child = Node()
    child.N = np.random.randint(1, 100)  # Random visit count for each child
    child.W = np.random.rand() * 100  # Random total value for each child
    child.P = np.random.rand()  # Random prior probability for each child
    dummy_node.children[f"move_{i}"] = child

# Benchmark the select_child function
pb_c_base = 19652
pb_c_init = 1.25
pb_c_factor = 1.0
fpu = 0.0#bestmove, bestchild = select_child(dummy_node, pb_c_base, pb_c_init, pb_c_factor, fpu)

num_runs = 1000000
# Benchmark the select_child function using timeit
benchmark_time = timeit.timeit(
    "select_child(dummy_node, pb_c_base, pb_c_init, pb_c_factor, fpu)",
    globals=globals(),
    number=num_runs
)

print(f"Average time taken over {num_runs} runs: {benchmark_time / num_runs} seconds")
print(f"Total time taken: {benchmark_time} seconds")

Average time taken over 1000000 runs: 1.9494292420004056e-06 seconds
Total time taken: 1.9494292420004058 seconds


In [6]:
from mcts.c import select_child

# Create a dummy node with 30 fake children
dummy_node = Node()
dummy_node.N = 100  # Set parent visit count

for i in range(30):
    child = Node()
    child.N = np.random.randint(1, 100)  # Random visit count for each child
    child.W = np.random.rand() * 100  # Random total value for each child
    child.P = np.random.rand()  # Random prior probability for each child
    dummy_node.children[f"move_{i}"] = child

# Benchmark the select_child function
pb_c_base = 19652
pb_c_init = 1.25
pb_c_factor = 1.0
fpu = 0.0#bestmove, bestchild = select_child(dummy_node, pb_c_base, pb_c_init, pb_c_factor, fpu)

num_runs = 1000000
# Benchmark the select_child function using timeit
benchmark_time = timeit.timeit(
    "select_child(dummy_node, pb_c_base, pb_c_init, pb_c_factor, fpu)",
    globals=globals(),
    number=num_runs
)

print(f"Average time taken over {num_runs} runs: {benchmark_time / num_runs} seconds")
print(f"Total time taken: {benchmark_time} seconds")

Average time taken over 1000000 runs: 5.751467379999667e-06 seconds
Total time taken: 5.751467379999667 seconds


In [4]:
# Update and remove vloss
def backup(search_path, value):
    for node in reversed(search_path):
        node.N = node.N - node.vloss + 1
        node.W = node.W + node.vloss + value
        node.vloss = 0
        value = flip_value(value)

# Create a fake search path with at least 30 elements
search_path = [Node() for _ in range(30)]
value = np.random.rand()

# Benchmark the backup function using timeit
backup_time = timeit.timeit(
    "backup(search_path, value)",
    globals=globals(),
    number=num_runs
)

print(f"Average time taken over {num_runs} runs: {backup_time / num_runs} seconds")
print(f"Total time taken: {backup_time} seconds")

Average time taken over 1000000 runs: 6.049426947000029e-06 seconds
Total time taken: 6.049426947000029 seconds


In [6]:
from mcts.c import backup

# Create a fake search path with at least 30 elements
search_path = [Node() for _ in range(30)]
value = np.random.rand()

# Benchmark the backup function using timeit
backup_time = timeit.timeit(
    "backup(search_path, value)",
    globals=globals(),
    number=num_runs
)

print(f"Average time taken over {num_runs} runs: {backup_time / num_runs} seconds")
print(f"Total time taken: {backup_time} seconds")

Average time taken over 1000000 runs: 4.521437020999656e-06 seconds
Total time taken: 4.521437020999656 seconds


In [7]:
import timeit

# Sample list
sample_list = list(range(1000))

# Method 1: Using reversed()
def traverse_reversed(lst):
    for item in reversed(lst):
        pass

# Method 2: Using slicing
def traverse_slicing(lst):
    for item in lst[::-1]:
        pass

# Method 3: Using a for loop with range
def traverse_range(lst):
    for i in range(len(lst)-1, -1, -1):
        item = lst[i]

# Number of runs for benchmarking
num_runs = 100000

# Benchmark each method
time_reversed = timeit.timeit('traverse_reversed(sample_list)', globals=globals(), number=num_runs)
time_slicing = timeit.timeit('traverse_slicing(sample_list)', globals=globals(), number=num_runs)
time_range = timeit.timeit('traverse_range(sample_list)', globals=globals(), number=num_runs)

print(f"Average time taken by reversed() over {num_runs} runs: {time_reversed / num_runs} seconds")
print(f"Average time taken by slicing over {num_runs} runs: {time_slicing / num_runs} seconds")
print(f"Average time taken by range over {num_runs} runs: {time_range / num_runs} seconds")

Average time taken by reversed() over 100000 runs: 6.639731540008142e-06 seconds
Average time taken by slicing over 100000 runs: 7.250656179994621e-06 seconds
Average time taken by range over 100000 runs: 2.4007662889998754e-05 seconds


In [3]:
from rchess import Cache
import time

In [5]:
cache = Cache()
num_items = 100

start_time = time.time()
for i in range(num_items):
    cache.set(i, i)
end_time = time.time()

print(f"Time taken to insert {num_items} items: {end_time - start_time} seconds")

Time taken to insert 100 items: 0.00013494491577148438 seconds


In [43]:
import multiprocessing
from multiprocessing.managers import BaseManager
import random
import time

def insert_numbers(cache, num_items):
    for _ in range(num_items):
        num = random.randint(1, 5)
        cache.set(num, num)

def read_numbers(cache, num_items, hit_counter):
    for _ in range(num_items):
        num = random.randint(1, 10)
        if cache.get(num) is not None:
            hit_counter.value += 1

if __name__ == "__main__":
    BaseManager.register('Cache', Cache)
    manager = BaseManager()
    manager.start()
    cache = manager.Cache()
    num_items = 100
    hit_counter1 = multiprocessing.Value('i', 0)
    hit_counter2 = multiprocessing.Value('i', 0)

    p1 = multiprocessing.Process(target=insert_numbers, args=(cache, num_items))
    p2 = multiprocessing.Process(target=read_numbers, args=(cache, num_items, hit_counter1))
    p3 = multiprocessing.Process(target=read_numbers, args=(cache, num_items, hit_counter2))

    start_time = time.time()

    p1.start()
    p2.start()
    p3.start()

    p1.join()
    p2.join()
    p3.join()

    end_time = time.time()

    print(f"Time taken: {end_time - start_time} seconds")
    print(f"Cache hits by process 2: {hit_counter1.value}")
    print(f"Cache hits by process 3: {hit_counter2.value}")

Time taken: 0.0209195613861084 seconds
Cache hits by process 2: 51
Cache hits by process 3: 51


In [2]:
%load_ext cython

In [ ]:
%%cython

#cython: language_level=3

import numpy as np
import timeit
from numba import jit
cimport numpy as cnp

def sum_array_cy(cnp.ndarray[cnp.float64_t, ndim=1] arr):
    cdef double result = 0
    for i in range(arr.size):
        result += arr[i]
    return result

def sum_np(cnp.ndarray[cnp.float64_t, ndim=1] arr):
    return np.sum(arr)

""" @jit(nopython=True)
def sum_jit(cnp.ndarray[cnp.float64_t, ndim=1] arr):
    return np.sum(arr) """

# Create a sample numpy array
arr = np.random.rand(1000000)

# Number of runs for benchmarking
num_runs = 1000

# Benchmark the sum_array_cy function
time_cy = timeit.timeit('sum_array_cy(arr)', globals=globals(), number=num_runs)

# Benchmark the sum_np function
time_np = timeit.timeit('sum_np(arr)', globals=globals(), number=num_runs)

# Benchmark the sum_jit function
#time_jit = timeit.timeit('sum_jit(arr)', globals=globals(), number=num_runs)

print(f"Average time taken by sum_array_cy over {num_runs} runs: {time_cy / num_runs} seconds")
print(f"Average time taken by sum_np over {num_runs} runs: {time_np / num_runs} seconds")
#print(f"Average time taken by sum_jit over {num_runs} runs: {time_jit / num_runs} seconds")

Content of stderr:
In file included from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1948,
                 from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /home/tomaz/.cache/ipython/cython/_cython_magic_a1c785493564886693c0306cd5c114742d03edac.c:1221:
/home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~Average time taken by sum_array_cy over 10000 runs: 0.0007097622633998981 seconds
Average time taken by sum_np over 10000 runs: 0.0001704781832999288 seconds
